In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [8]:
from llama_index.llms.gemini import Gemini

import google.generativeai as genai

from llama_index.core import SimpleDirectoryReader

from llama_index.core import VectorStoreIndex

from IPython.display import Markdown, display

from llama_index.core import ServiceContext

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.embeddings.gemini import GeminiEmbedding

/Users/lukaszmichalik/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
genai.configure(api_key=GOOGLE_API_KEY)

In [10]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(f"Model: {models.name}")

Model: models/gemini-1.0-pro-vision-latest
Model: models/gemini-pro-vision
Model: models/gemini-1.5-pro-latest
Model: models/gemini-1.5-pro-002
Model: models/gemini-1.5-pro
Model: models/gemini-1.5-flash-latest
Model: models/gemini-1.5-flash
Model: models/gemini-1.5-flash-002
Model: models/gemini-1.5-flash-8b
Model: models/gemini-1.5-flash-8b-001
Model: models/gemini-1.5-flash-8b-latest
Model: models/gemini-2.5-pro-preview-03-25
Model: models/gemini-2.5-flash-preview-04-17
Model: models/gemini-2.5-flash-preview-05-20
Model: models/gemini-2.5-flash
Model: models/gemini-2.5-flash-preview-04-17-thinking
Model: models/gemini-2.5-flash-lite-preview-06-17
Model: models/gemini-2.5-pro-preview-05-06
Model: models/gemini-2.5-pro-preview-06-05
Model: models/gemini-2.5-pro
Model: models/gemini-2.0-flash-exp
Model: models/gemini-2.0-flash
Model: models/gemini-2.0-flash-001
Model: models/gemini-2.0-flash-lite-001
Model: models/gemini-2.0-flash-lite
Model: models/gemini-2.0-flash-lite-preview-02-05


In [11]:
documents = SimpleDirectoryReader('../data').load_data()

In [12]:
documents

[Document(id_='888fdc71-3c7c-4cba-b418-32790c317329', embedding=None, metadata={'file_path': '/Users/lukaszmichalik/Documents/GitHub/MICHALIK/Retrieval-Augmented-Generation/07_QA_System_with_Gemini/notebook/../data/rowdata.txt', 'file_name': 'rowdata.txt', 'file_type': 'text/plain', 'file_size': 39028, 'creation_date': '2025-07-02', 'last_modified_date': '2025-07-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally 

In [13]:
model = Gemini(models='gemini-1.5-pro', api_key=GOOGLE_API_KEY)

/var/folders/fw/fg0_f2fx5tn55v9k_2zxd6d00000gn/T/ipykernel_10106/787923438.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model = Gemini(models='gemini-1.5-pro', api_key=GOOGLE_API_KEY)


In [22]:
# Split the documents into smaller chunks to avoid exceeding the payload size limit
from llama_index.core import Document

# Define a function to split large documents into smaller chunks
def split_documents(documents, max_chunk_size=1000):
	split_docs = []
	for doc in documents:
		content = doc.text
		for i in range(0, len(content), max_chunk_size):
			chunk = content[i:i + max_chunk_size]
			split_docs.append(Document(text=chunk, metadata=doc.metadata))
	return split_docs

# Split the documents
split_docs = split_documents(documents)

# Use the smaller chunks for embedding
gemini_embed_model = GeminiEmbedding(model_name='models/embedding-001', nodes=split_docs)

/var/folders/fw/fg0_f2fx5tn55v9k_2zxd6d00000gn/T/ipykernel_10106/774072558.py:18: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model = GeminiEmbedding(model_name='models/gemini-1.5-prof', nodes=split_docs)


In [15]:
!pip show llama-index

Name: llama-index
Version: 0.12.45
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License-Expression: MIT
Location: /Users/lukaszmichalik/.local/lib/python3.9/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [16]:
# Install or update llama-index to latest version
%pip install -U llama-index

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.core import Settings

Settings.embed_model = gemini_embed_model

In [23]:
index = VectorStoreIndex.from_documents(split_docs, embed_model=gemini_embed_model)

NotFound: 404 models/gemini-1.5-prof is not found for API version v1beta, or is not supported for embedContent. Call ListModels to see the list of available models and their supported methods.